# **팀명 : 678**
# **Prediction Code**



# Library 설치 및 데이터 불러오기

In [ ]:
# Library 설치 및 데이터 불러오기

# konlpy Mecab 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

# 데이터 불러오기
import pandas as pd
path = "/content/678/"
test = pd.read_csv(path+"1.Data/news_test.csv",encoding = 'utf-8-sig')
submission = pd.read_csv(path+"1.Data/sample_submission.csv",encoding = 'utf-8-sig')

# 시간 측정 시작

In [ ]:
import time
start = time.time()

# Library 불러오기

In [ ]:
import numpy as np
from tensorflow import keras
from konlpy.tag import Mecab
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

# Tokenizer, Model 불러오기

In [ ]:
vectorizer = joblib.load(path+"3.Tokenizer/vectorizer.sav") # tokenizer 불러오기
lstm_model = keras.models.load_model(path+"5.Model/lstm_model.h5") # lstm 모델 불러오기
lgbm_model = joblib.load(path+'5.Model/lgbm_model.pkl') # lgbm 모델 불러오기

# 변수추가 + 형태소 분석 + 전처리

In [ ]:
# feature 추가

# content 맨앞이 "[" / "(" / "제목" 이면 info 1
test["content_startswith_["]=test.content.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(") or str(x).startswith("제목"))+0

# title에 해당 단어들이 포함되면 info 1
title_noise = ['적중 100%', '글로벌 주요 뉴스', '[전문가 의견]', 
               '[포커스]', '※','■', '★',' TOP', 'BEST',
               '전문가의 눈', '전문가선정', '전문가의견','】','후속주도 감사합니다',
               '전문가추천', '주요이슈']
def title_choose(x):
  if ("종목" in x[-6:]) or ("관련주" in x[-5:]):
    return 1
  for noise in title_noise:
    if noise in x.upper():
        return 1
  return 0
test["info1_title"]=test['title'].apply(title_choose)

# content에 해당 단어들이 포함되면 info 1
content_noise = ['00%', '긴급공개', '긴급 공개','임상3상','# ','대장株','대장주','카톡','원"만','TOP','BEST']
def content_choose(x):
  if (x=='관련기사') or (x=="관련 테마분석") or (x=="코스피") or (x=="코스닥"):
    return 1
  for noise in content_noise:
    if noise in x.upper():
        return 1
  return 0
test["info1_content"]=test["content"].apply(content_choose)

# order을 이용한 feature
title_group = (test.groupby(["title"]).count())["n_id"]
test["new_ord"]=test.apply(lambda x: x["ord"]/title_group[x["title"]], axis=1)

In [ ]:
# 형태소 분석 및 전처리
def text_preprocessing(text_list):
    tokenizer = Mecab() #형태소 분석기

    token_list = [] 
    
    for text in text_list:
        txt = re.sub("[a-zA-Z0-9]", ' ', text) #영문, 숫자 제거 -> 특수문자는 제거하지 않음
        txt = re.sub('[가-힣]+\s기자','기자', txt) #기자 이름 제거
        token = tokenizer.morphs(txt) #형태소 분석

        token = [t for t in token] 
        token_list.append(token)
        
    return token_list, tokenizer

test['new_article'], mecab = text_preprocessing(test['content'])

In [ ]:
# vectorizer
test_X_seq = vectorizer.texts_to_sequences(test["new_article"])
test_X = pad_sequences(test_X_seq, maxlen = 100) # 길이를 맞춰줌

# 새로 생성한 변수와 vectorizer을 concat
test_X = np.concatenate([test_X,test[["info1_title","info1_content","new_ord","content_startswith_["]].values.reshape(-1,4)], axis=1)

# test 예측

In [ ]:
# lstm 예측값
predicted = lstm_model.predict([test_X[:,:100],test_X[:,-4:]], batch_size = 100)

# 해당 문장의 예측값과 해당문장의 앞문장, 앞앞문장의 예측값을 feature로 하여 최종 예측 -> lgbm 이용
len_predicted = len(predicted)
pre_predicted1 = np.array([predicted[idx-1][0] for idx in range(len(predicted))]).reshape(-1,1) # 앞문장 예측값
pre_predicted2 = np.array([predicted[idx-2][0] for idx in range(len(predicted))]).reshape(-1,1) # 앞앞문장 예측값
test["predicted"]=predicted
test["pre_predicted1"]=pre_predicted1
test["pre_predicted2"]=pre_predicted2

lgbm_final_predicted = lgbm_model.predict(test[["predicted","pre_predicted1","pre_predicted2"]])
test["info"]=lgbm_final_predicted

# ord가 1,2인 것은 lgbm모델을 위한 feature가 올바르지 않기 때문에 lstm모델 예측값만을 통해 예측(threshold 0.6)
test["info"][test["ord"]==1]=(test["predicted"][test["ord"]==1]>=0.6).astype("int")
test["info"][test["ord"]==2]=(test["predicted"][test["ord"]==2]>=0.6).astype("int")

# 사이트와 결측값의 경우 rule 기반 예측
test["info"][test["content"].apply(lambda x: True if ('http://etoday.bujane.co.kr/' in x) or ('http://www.hisl.co.kr/0306/' == x)  or (x==']]') else False)]=1

# submission 파일 info에 반영
submission["info"]=test['info']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
print(time.time() - start)

# 제출

In [ ]:
# 최종 submission file 제출
submission.to_csv("/content/submission.csv", encoding="utf-8-sig",index=False)